# Extracción BD Colombianos registrados en el exterior

In [1]:
#Importar Librerias necesarias
import dotenv
import os
import pandas as pd
import sodapy
from sodapy import Socrata

In [2]:
# Carga las variables de entorno
dotenv.load_dotenv()

# Obtiene el valor de la variable API_KEY
api_key = os.getenv("API_KEY")
api_id = os.getenv("API_ID")
api_name = os.getenv("API_NAME")

# Example authenticated client (needed for non-public datasets):
client = sodapy.Socrata("www.datos.gov.co",
                 api_name,
                 username=api_id,
                 password=api_key)

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("www.datos.gov.co", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                 api_name,
                 username=api_id,
                 password=api_key)

# Parámetros de búsqueda
dataset_id = "y399-rzwf"
limit_per_request = 1000

# Lista para almacenar los resultados
all_results = []

# Realiza solicitudes hasta que ya no haya más resultados
offset = 0

In [4]:
# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("y399-rzwf", limit=2000)

while True:
    # Realiza una solicitud para obtener un lote de resultados
    results = client.get(dataset_id, limit=limit_per_request, offset=offset)
    
    # Si no hay más resultados, detén el bucle
    if not results:
        break
    
    # Agrega los resultados al conjunto total
    all_results.extend(results)
    
    # Actualiza el valor de offset para la próxima solicitud
    offset += limit_per_request

In [5]:
# Convierte los resultados en un DataFrame de pandas
results_df = pd.DataFrame.from_records(all_results)

In [6]:
results_df.to_csv('data/raw/out.csv', index=False)

# Exploración

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/raw/out.csv')

In [3]:
df.columns

Index(['pa_s', 'c_digo_iso_pa_s', 'oficina_de_registro', 'grupo_edad',
       'edad_a_os', 'rea_conocimiento', 'sub_area_conocimiento',
       'nivel_acad_mico', 'estado_civil', 'g_nero', 'etnia_de_la_persona',
       'estatura_cm', 'localizaci_n', 'cantidad_de_personas'],
      dtype='object')

In [4]:
df.shape

(708454, 14)

In [5]:
df.drop(columns=['c_digo_iso_pa_s',
                'oficina_de_registro',
                # 'estatura_cm',  Creo que esta no es relevante
                'localizaci_n', 
                'cantidad_de_personas'], inplace=True)

In [6]:
df.columns

Index(['pa_s', 'grupo_edad', 'edad_a_os', 'rea_conocimiento',
       'sub_area_conocimiento', 'nivel_acad_mico', 'estado_civil', 'g_nero',
       'etnia_de_la_persona', 'estatura_cm'],
      dtype='object')

In [7]:
df = df.astype({'pa_s':'category', 
                'grupo_edad':'category', 
                'edad_a_os':'int', 
                'rea_conocimiento':'category',
                'sub_area_conocimiento':'category', 
                'nivel_acad_mico':'category', 
                'estado_civil':'category', 
                'g_nero':'category',
                'etnia_de_la_persona':'category', 
                'estatura_cm':'int'})

In [8]:
df.dtypes

pa_s                     category
grupo_edad               category
edad_a_os                   int64
rea_conocimiento         category
sub_area_conocimiento    category
nivel_acad_mico          category
estado_civil             category
g_nero                   category
etnia_de_la_persona      category
estatura_cm                 int64
dtype: object

In [9]:

# Contar la frecuencia de cada categoría
frequencies = df['pa_s'].value_counts()

# Normalizar las categorías
normalized_data = frequencies / frequencies.sum()

In [10]:
df['pais_normalizada'] = df['pa_s'].map(normalized_data)